# Model Comparison

## Imports

In [1]:
# Custom
import sys
sys.path.append('../')
from utils.dataset_manager import fit_dataset, get_classes_weights
from utils.constant import ATTACKS, ALL_ATTACKS, FEATURES, LABELS

# Models
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report

# Other
import warnings
from tqdm import tqdm

# Ignore warnings
warnings.filterwarnings('ignore')

## Dataset

In [4]:
n_files = 2

df_train, df_test = fit_dataset(n_files, ALL_ATTACKS)

# Binary classification: Bening traffic vs. DDoS traffic
df_train = df_train[(df_train[LABELS] == ALL_ATTACKS['DDoS-SYN_Flood']) | (df_train[LABELS] == ALL_ATTACKS['BenignTraffic'])]
df_test = df_test[(df_test[LABELS] == ALL_ATTACKS['DDoS-SYN_Flood']) | (df_test[LABELS] == ALL_ATTACKS['BenignTraffic'])]

df_train[LABELS] = df_train[LABELS].apply(lambda x: 1 if x == ALL_ATTACKS['DDoS-SYN_Flood'] else 0)
df_test[LABELS] = df_test[LABELS].apply(lambda x: 1 if x == ALL_ATTACKS['DDoS-SYN_Flood'] else 0)

X_train, y_train = df_train[FEATURES], df_train[LABELS]
X_test, y_test = df_test[FEATURES], df_test[LABELS]

# Prints
print('Training Population: {}'.format(len(df_train)))
print('Testing Population: {}'.format(len(df_test)))

100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


Training Population: 50774
Testing Population: 30550


In [5]:
get_classes_weights(df_train)

{1: 0.6350878070745984, 0: 2.350648148148148}

## Models

Since we have unbalanced data, we have to add their weigth in each model

In [4]:
models = {
    'log_reg': LogisticRegression(class_weight=get_classes_weights(df_train)),
    'xgb': XGBClassifier(scale_pos_weight=sum(y_train == 0) / sum(y_train == 1)),
    #'svm': SVC(), # Too slow
}

### Training

In [5]:
for model in tqdm(models):
    models[model].fit(X_train, y_train)

100%|██████████| 2/2 [00:01<00:00,  1.10it/s]


## Evaluation

In [6]:
# Predict
for model in tqdm(models):
    y_pred = list(models[model].predict(X_test))

    # Evaluate
    y_test = list(y_test)
    print('Model: ', model)
    print('  accuracy_score = ', accuracy_score(y_pred, y_test))
    print('  recall_score = ', recall_score(y_pred, y_test, average='macro'))
    print('  precision_score = ', precision_score(y_pred, y_test, average='macro'))
    print('  f1_score = ', f1_score(y_pred, y_test, average='macro'))
    print('  classification_report = \n', classification_report(y_pred, y_test))

  0%|          | 0/2 [00:00<?, ?it/s]

Model:  log_reg
  accuracy_score =  0.999050736497545
  recall_score =  0.9979097813456863
  precision_score =  0.9992271352927968


 50%|█████     | 1/2 [00:00<00:00,  2.74it/s]

  f1_score =  0.9985667335407329
  classification_report = 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6410
           1       1.00      1.00      1.00     24140

    accuracy                           1.00     30550
   macro avg       1.00      1.00      1.00     30550
weighted avg       1.00      1.00      1.00     30550

Model:  xgb
  accuracy_score =  0.9998036006546644


100%|██████████| 2/2 [00:00<00:00,  3.02it/s]

  recall_score =  0.9995881286891597
  precision_score =  0.9998182520117515
  f1_score =  0.9997031379513762
  classification_report = 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6391
           1       1.00      1.00      1.00     24159

    accuracy                           1.00     30550
   macro avg       1.00      1.00      1.00     30550
weighted avg       1.00      1.00      1.00     30550

